In [7]:
import ollama
from ollama import chat

from huggingface_hub import snapshot_download

from docling.datamodel.pipeline_options import PdfPipelineOptions, RapidOcrOptions, PictureDescriptionApiOptions
from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend
from docling.datamodel.base_models import InputFormat
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.chunking import HybridChunker
from docling_core.types.doc import PictureItem, TableItem, TextItem
from docling_core.types.doc.labels import DocItemLabel

from pydantic.v1 import BaseModel

from transformers import AutoTokenizer
from sentence_transformers import CrossEncoder

from BM25 import load_bm25, create_bm25, bm25_search
from util.docling_util import *
from util.qdrant_util import qdrant_DBConnector, DataObject
from util.text_splitter import RecursiveTextSplitter, DataFrameFormatter
from util.ollama_util import *

import os
import json
from pathlib import Path

import uuid

# Configure Settings

In [8]:
do_ocr = False
do_image_summary = True

kb_name = "我的kb"
kb_id = str(uuid.uuid4())[:8]
new_kb_name = f"{kb_name}_{kb_id}"

In [9]:
# document local path or URL
doc = "./test_doc/國泰人壽祿美富利率變動型美元終身壽險（定期給付型）.pdf" # simple text with a image and table
doc1 = "./test_doc/吉美利101利率變動型美元終身壽險(定期給付型)DM.pdf" # complex text with simple aesthetic table
doc2 = "./test_doc/美利雙寶利率變動型美元終身保險 (定期給付型)DM.pdf" # complex text with complex aesthetic table
doc3 = "./test_doc/國泰金控出勤管理須知(修正後).pdf" # simple text with a simple table
doc4 = "./test_doc/國泰金控員工國內出差要點.pdf" # simple text with a simple table
doc5 = "./test_doc/國泰金控員工國外出差要點.pdf" # simple text with complex table
doc6 = "./test_doc/國泰金融控股股份有限公司資訊安全管理辦法.pdf" # simple text
doc7 = "./test_doc/國泰金融控股股份有限公司_後台維護系統管理作業要點_F.pdf" # flow map with blank table
doc8 = "./test_doc/國泰金融控股股份有限公司金融科技創新業務管理辦法_20240125.pdf" # flow map
doc9 = "./test_doc/國泰金控暨子公司「對話機器人 - 阿發」視覺形象管理辦法.pdf" # complex figure design doc
doc10 = "./test_doc/國泰醫院113年工作計畫.pdf" # cross page table and HAS index, scanned document OCR needed
doc11 = "./test_doc/國泰醫院113年經費預算.pdf" # large number table, scanned document OCR needed
doc12 = "./test_doc/國泰醫院112年工作報告.pdf" # table with check board inside, scanned document OCR needed
doc13 = "./test_doc/112年國泰醫療財團法人財務報告.pdf" # loose structured table, scanned document OCR needed
doc14 = "./test_doc/111年4-6月愛心捐款及捐贈物資徵信名冊.pdf" #compacted table
doc15 = "./test_doc/畢業學分審核表.pdf" # vertical table

# ollama embedding model
embed_model = "bge-m3:latest"

# reranker 
rerank_model = CrossEncoder('BAAI/bge-reranker-v2-m3', max_length=1024)

# ollama llm
#llm = "deepseek-r1:14b-max-context"
llm = "deepseek-r1:14b-quarter-context"
#llm = "deepseek-r1:14b-eighth-context"

if do_ocr:
    # PyPdfium with RapidOCR
    # ----------------------
    # Download RappidOCR models from HuggingFace
    print("Downloading RapidOCR models")
    download_path = snapshot_download(repo_id="SWHL/RapidOCR")

    det_model_path = os.path.join(
        download_path, "PP-OCRv4", "ch_PP-OCRv4_det_infer.onnx"
    )
    rec_model_path = os.path.join(
        download_path, "PP-OCRv4", "ch_PP-OCRv4_rec_infer.onnx"
    )
    cls_model_path = os.path.join(
        download_path, "PP-OCRv3", "ch_ppocr_mobile_v2.0_cls_train.onnx"
    )
    ocr_options = RapidOcrOptions(
        det_model_path=det_model_path,
        rec_model_path=rec_model_path,
        cls_model_path=cls_model_path,
        #force_full_page_ocr=True
    )

    pipeline_options = PdfPipelineOptions()

    pipeline_options.do_ocr = True
    pipeline_options.do_table_structure = True
    pipeline_options.table_structure_options.do_cell_matching = True
    pipeline_options.table_structure_options.mode = 'accurate'

    pipeline_options.images_scale = 2.0
    pipeline_options.generate_page_images = True
    pipeline_options.generate_picture_images = True

    pipeline_options.ocr_options = ocr_options

    pypdfium_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(
            pipeline_options=pipeline_options, backend=PyPdfiumDocumentBackend
            )
    }
    )

else:
    # PyPdfium without EasyOCR
    # --------------------
    pipeline_options = PdfPipelineOptions()
    pipeline_options.do_ocr = False
    pipeline_options.do_table_structure = True
    pipeline_options.table_structure_options.do_cell_matching = False
    #pipeline_options.table_structure_options.do_cell_matching = True

    #pipeline_options.table_structure_options.mode = 'accurate'

    pipeline_options.images_scale = 2
    pipeline_options.generate_page_images = True
    pipeline_options.generate_picture_images = True

    pypdfium_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(
            pipeline_options=pipeline_options, backend=PyPdfiumDocumentBackend
            )
        }
    )

# print json
def show_json(data):
    if isinstance(data, str):
        obj = json.loads(data)
        print(json.dumps(obj, indent=4, ensure_ascii=False))
    elif isinstance(data, dict) or isinstance(data, list):
        print(json.dumps(data, indent=4, ensure_ascii=False))
    elif issubclass(type(data), BaseModel):
        print(json.dumps(data.dict(), indent=4, ensure_ascii=False))

In [10]:
UPLOAD_FOLDER = './test_fake_uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

def save2fakeuploads(doc_paths):
    new_doc_sources = []
    for path in doc_paths:
        file_name = os.path.basename(path)
        basename, file_ext = os.path.splitext(file_name)

        safe_basename = ''.join(c for c in basename if c.isalnum() or '\u4e00' <= c <= '\u9fff')[:30]
        file_id = str(uuid.uuid4())[:8]
        new_filename = f"{safe_basename}_{file_id}{file_ext}"
        file_path = os.path.join(UPLOAD_FOLDER, new_filename)
        with open(path, "rb") as f_in, open(file_path, "wb") as f_out:
            f_out.write(f_in.read())
        new_doc_sources.append(file_path)
    return new_doc_sources

# Docling Converter & Patched HybridChunker

In [11]:
#doc_source = [doc15]                                       # vertical table

#doc_source = [doc, doc7, doc8]                             # image caption required
#doc_source = [doc1, doc2, doc3, doc4, doc5, doc6, doc14]   # text and table
#doc_source = [doc10, doc11, doc12, doc13]                  # OCR required

#doc_source = [doc5, doc7] 
#doc_source = [doc4, doc8] 
doc_source = [doc7] 

new_doc_sources = save2fakeuploads(doc_source)

conv_results = pypdfium_converter.convert_all(
    new_doc_sources,
    #doc_source,
    raises_on_error=True,  # to let conversion run through all and examine results at the end
)

conv_results_list = list(conv_results)

# Do hybrid chunking to merge similar chunk
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")


In [12]:
output_dir = Path("figure_storage")
output_dir.mkdir(parents=True, exist_ok=True)

# Save images of figures and tables for later summary reference
for conv_res in conv_results_list:
    docling_docs = conv_res.document
    doc_filename = conv_res.input.file.stem
    table_counter = 0
    picture_counter = 0
    
    for element, _level in docling_docs.iterate_items():
        if isinstance(element, TableItem):
            table_counter += 1
            element_image_filename = (
                output_dir / f"{doc_filename}-table-{table_counter}.png"
            )
            with element_image_filename.open("wb") as fp:
                element.get_image(conv_res.document).save(fp, "PNG")

        if isinstance(element, PictureItem):
            picture_counter += 1
            element_image_filename = (
                output_dir / f"{doc_filename}-picture-{picture_counter}.png"
            )
            with element_image_filename.open("wb") as fp:
                element.get_image(docling_docs).save(fp, "PNG")

if do_image_summary:
    hybrid_chunker = HybridChunker(
        tokenizer=tokenizer,
        max_tokens=8000,
        serializer_provider=ImgAnnotationSerializerProvider(),
    )
else:
    hybrid_chunker = HybridChunker(
        tokenizer=tokenizer,
        max_tokens=8000,
        merge_peers=True  # optional, defaults to True
    )
    
# text chunks
all_chunks = []
for conv_res in conv_results_list:
    docling_docs = conv_res.document
    chunk_iter = hybrid_chunker.chunk(dl_doc=docling_docs)
    chunks = list(chunk_iter)
    all_chunks += chunks

In [13]:
for i, chunk in enumerate(all_chunks[:]):
    print(f"=== {i} ===")
    txt_tokens = len(tokenizer.tokenize(chunk.text))
    print(f"chunk.text ({txt_tokens} tokens):\n{repr(chunk.text)}")

    ser_txt = hybrid_chunker.serialize(chunk=chunk)
    ser_tokens = len(tokenizer.tokenize(ser_txt))
    print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{repr(ser_txt)}")
    #print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{ser_txt}")

    print(chunk.meta)

    print()

=== 0 ===
chunk.text (61 tokens):
'中華民國 108 年 10 月 05 日5 日訂日訂定 中華民國 111 年 10 月 13 日3 日修日修訂\n權責單位：數：數位數據暨據暨科暨科技發技發展中心數心數位生態發展部'
chunker.serialize(chunk) (77 tokens):
'國泰優惠 APP 後P 後台維護系統管理要點\n中華民國 108 年 10 月 05 日5 日訂日訂定 中華民國 111 年 10 月 13 日3 日修日修訂\n權責單位：數：數位數據暨據暨科暨科技發技發展中心數心數位生態發展部'
schema_name='docling_core.transforms.chunker.DocMeta' version='1.0.0' doc_items=[DocItem(self_ref='#/texts/1', parent=RefItem(cref='#/body'), children=[], content_layer=<ContentLayer.BODY: 'body'>, label=<DocItemLabel.TEXT: 'text'>, prov=[ProvenanceItem(page_no=1, bbox=BoundingBox(l=366.64801025390625, t=724.0700073242188, r=541.9409790039062, b=692.9979858398438, coord_origin=<CoordOrigin.BOTTOMLEFT: 'BOTTOMLEFT'>), charspan=(0, 55))]), DocItem(self_ref='#/texts/2', parent=RefItem(cref='#/body'), children=[], content_layer=<ContentLayer.BODY: 'body'>, label=<DocItemLabel.TEXT: 'text'>, prov=[ProvenanceItem(page_no=1, bbox=BoundingBox(l=260.54998779296875, t=675.219970703125, r=541.2220458984375, b=664.167968

/var/folders/v7/_g3b28s51m532s03dhwffy9c0000gn/T/ipykernel_60706/2500604497.py:6: DeprecationWarning: Use contextualize() instead.
  ser_txt = hybrid_chunker.serialize(chunk=chunk)


# Rechunk large chunks with RecursiveTextSplitter

In [14]:
big_chunk = []

for i, chunk in enumerate(all_chunks[:]):
    if type(chunk) == tuple:
        ser_txt = chunk[0]
    else:
        ser_txt = hybrid_chunker.serialize(chunk=chunk)
    ser_tokens = len(tokenizer.tokenize(ser_txt))
    if ser_tokens > 1024:
        print(f"=== {i} ===")
        #print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{repr(ser_txt)}")
        print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{ser_txt}")
        big_chunk.append(chunk)
        print()
    else:
        pass

/var/folders/v7/_g3b28s51m532s03dhwffy9c0000gn/T/ipykernel_60706/499157167.py:7: DeprecationWarning: Use contextualize() instead.
  ser_txt = hybrid_chunker.serialize(chunk=chunk)


In [15]:
splitter = RecursiveTextSplitter(tokenizer=tokenizer, max_tokens=1024, overlap=150, min_length_ratio=1)

rechunked_large_chunks = []

for i, chunk in enumerate(big_chunk):
    if type(chunk) == tuple:
        ser_txt = chunk[0]
    else:
        ser_txt = hybrid_chunker.serialize(chunk=chunk)
        
    if splitter.tokenize_len(ser_txt) > splitter.max_tokens:
        sub_chunks = splitter.split_text(ser_txt)
        #sub_chunks_meta = {"filename": chunk.meta.origin.filename}
        sub_chunks_meta = extract_meta_from_docling(chunk.meta)
        sub_chunks_pair = [(text, sub_chunks_meta) for text in sub_chunks]
        rechunked_large_chunks.extend(sub_chunks_pair)
    else:
        rechunked_large_chunks.append(ser_txt)

In [16]:
for i, chunk in enumerate(rechunked_large_chunks[:]):
    
    ser_txt = chunk[0]
    ser_tokens = len(tokenizer.tokenize(ser_txt))

    print(f"=== {i} ===")
    #print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{repr(ser_txt)}")
    print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{ser_txt}")
    print(chunk[1])

    print()

# Or table extraction result

In [17]:
# table chunks
all_tables = extract_tables(conv_results_list)

In [18]:
table_formatter = DataFrameFormatter(tokenizer=tokenizer, show_index=False, max_tokens=1024)

table_chunks = []
for table in all_tables:
    chunks = table_formatter.chunk_rows(table[0])
    chunks_pair = [(text, table[1]) for text in chunks]
    table_chunks.extend(chunks_pair)

In [19]:
for i, chunk in enumerate(table_chunks[:]):
    
    ser_txt = chunk[0]
    ser_tokens = len(tokenizer.tokenize(ser_txt))

    print(f"=== {i} ===")
    #print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{repr(ser_txt)}")
    print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{ser_txt}")
    print(chunk[1])
    print()

=== 0 ===
chunker.serialize(chunk) (10 tokens):
所屬單位 = 所屬單位




{'filename': '國泰金融控股股份有限公司後台維護系統管理作業要點F_4222c843', 'table_ref': ['國泰金融控股股份有限公司後台維護系統管理作業要點F_4222c843-table-1.png'], 'page_ref': [5]}



# Merge all chunks

In [20]:
all_chunks.extend(rechunked_large_chunks)
all_chunks.extend(table_chunks)
#all_chunks.extend(figure_summary_chunks)
all_chunks

[DocChunk(text='中華民國 108 年 10 月 05 日5 日訂日訂定 中華民國 111 年 10 月 13 日3 日修日修訂\n權責單位：數：數位數據暨據暨科暨科技發技發展中心數心數位生態發展部', meta=DocMeta(schema_name='docling_core.transforms.chunker.DocMeta', version='1.0.0', doc_items=[DocItem(self_ref='#/texts/1', parent=RefItem(cref='#/body'), children=[], content_layer=<ContentLayer.BODY: 'body'>, label=<DocItemLabel.TEXT: 'text'>, prov=[ProvenanceItem(page_no=1, bbox=BoundingBox(l=366.64801025390625, t=724.0700073242188, r=541.9409790039062, b=692.9979858398438, coord_origin=<CoordOrigin.BOTTOMLEFT: 'BOTTOMLEFT'>), charspan=(0, 55))]), DocItem(self_ref='#/texts/2', parent=RefItem(cref='#/body'), children=[], content_layer=<ContentLayer.BODY: 'body'>, label=<DocItemLabel.TEXT: 'text'>, prov=[ProvenanceItem(page_no=1, bbox=BoundingBox(l=260.54998779296875, t=675.219970703125, r=541.2220458984375, b=664.16796875, coord_origin=<CoordOrigin.BOTTOMLEFT: 'BOTTOMLEFT'>), charspan=(0, 33))])], headings=['國泰優惠 APP 後P 後台維護系統管理要點'], captions=None, origin=DocumentOrigin(mime

In [21]:
node_text, node_metadatas = [], []
for chunk in all_chunks:
    if type(chunk) == tuple:
        node_text.append(chunk[0])
        basename, _ = os.path.splitext(chunk[1]["filename"])
        file_id = basename.split('_')[-1]
        chunk[1]["file_id"] = file_id
        chunk[1]["kb_name"] = new_kb_name
        chunk[1]["kb_id"] = kb_id
        #node_metadatas.append(json.dumps(meta_dict, indent=4, ensure_ascii=False))
        node_metadatas.append(chunk[1])
    else:
        node_text.append(hybrid_chunker.contextualize(chunk=chunk))
        meta_dict = extract_meta_from_docling(chunk.meta)
        basename, _ = os.path.splitext(meta_dict.get("filename"))
        file_id = basename.split('_')[-1]
        meta_dict["file_id"] = file_id
        meta_dict["kb_name"] = new_kb_name
        meta_dict["kb_id"] = kb_id
        #node_metadatas.append(json.dumps(meta_dict, indent=4, ensure_ascii=False))
        node_metadatas.append(meta_dict)

In [22]:
for i in node_metadatas:
    print(i)
    print()

{'filename': '國泰金融控股股份有限公司後台維護系統管理作業要點F_4222c843.pdf', 'table_ref': [], 'image_ref': [], 'page_ref': [1], 'file_id': '4222c843', 'kb_name': '我的kb_98fde1f6', 'kb_id': '98fde1f6'}

{'filename': '國泰金融控股股份有限公司後台維護系統管理作業要點F_4222c843.pdf', 'table_ref': [], 'image_ref': [], 'page_ref': [1], 'file_id': '4222c843', 'kb_name': '我的kb_98fde1f6', 'kb_id': '98fde1f6'}

{'filename': '國泰金融控股股份有限公司後台維護系統管理作業要點F_4222c843.pdf', 'table_ref': [], 'image_ref': [], 'page_ref': [1], 'file_id': '4222c843', 'kb_name': '我的kb_98fde1f6', 'kb_id': '98fde1f6'}

{'filename': '國泰金融控股股份有限公司後台維護系統管理作業要點F_4222c843.pdf', 'table_ref': [], 'image_ref': [], 'page_ref': [1, 2], 'file_id': '4222c843', 'kb_name': '我的kb_98fde1f6', 'kb_id': '98fde1f6'}

{'filename': '國泰金融控股股份有限公司後台維護系統管理作業要點F_4222c843.pdf', 'table_ref': [], 'image_ref': [], 'page_ref': [2], 'file_id': '4222c843', 'kb_name': '我的kb_98fde1f6', 'kb_id': '98fde1f6'}

{'filename': '國泰金融控股股份有限公司後台維護系統管理作業要點F_4222c843.pdf', 'table_ref': [], 'image_ref': [], 'page_ref': [

In [29]:
node_text[-3]

'附件-、\n1. 摘要圖片的主要內容：\n圖片為國泰金融控股的後台維護系統管理流程圖，涉及申請、審核、承辦作業及驗收四個階段，詳細描述了從提出申請到驗收結案的處理流程。\n\n2. 圖像元素及概念：\n該圖包含流程框和決策菱形，展示了申請和審核中的各個步驟，並以箭頭連接代表流程順序。\n\n3. 重要概念、重點事項或關鍵字：\n- 申請：提出申請需求、申請人所屬單位主管簽核\n- 審核：生活金融科會審核、涉及會員個人資料、會辦資料、控資資訊部份派審核\n- 承辦作業：生活金融科帳號權限設定、新增代碼對應功能權限、設定帳號對應權限代碼\n- 驗收：驗收結案\n\n4. 結構化資訊描述：\n- 申請階段：包括提出申請需求及申請人所屬單位主管簽核。\n- 審核階段：生活金融科會進行審核，確認是否涉及會員個人資料，如涉及則會辦資料所屬單位簽核。\n- 承辦作業階段：進行生活金融科帳號權限設定，根據功能權限申請一致性進行操作。\n- 驗收階段：最後由相關部門進行驗收結案。'

In [19]:
embedded_text = []
for i in node_text:
    embedded_text.append(get_embeddings(i))

# Qdrant Connection & Retrieval


In [1]:
from qdrant_client import QdrantClient, models

qdrant_client = QdrantClient(host="localhost", port=6333)

scroll_result = qdrant_client.scroll(
    collection_name="qdrant_withID",
    limit=1000,
    scroll_filter=models.Filter(
        should=[
            models.FieldCondition(
                key="metadata.filename",
                match=models.MatchValue(value="國泰金控員工國外出差要點_c0a8eb7a.pdf")
            ),
            #models.FieldCondition(
            #    key="metadata.origin.filename",
            #    match=models.MatchValue(value="國泰金控員工國外出差要點.pdf")
            #)
        ]
    )
)
scroll_result

([], None)

In [4]:
from qdrant_client import QdrantClient, models

qdrant_client = QdrantClient(host="localhost", port=6333)

scroll_result = qdrant_client.scroll(
    collection_name="qdrant_withID",
    limit=1000,
    scroll_filter=models.Filter(
        must_not=[
            models.IsEmptyCondition(
                is_empty=models.PayloadField(key="metadata.table_ref"),
            ),
            models.IsEmptyCondition(
                is_empty=models.PayloadField(key="metadata.image_ref"),
            )
        ]
    ),
)
scroll_result

([], None)

delete_result = qdrant_client.delete(
    collection_name="qdrant_final",
    points_selector=models.FilterSelector(
        filter=models.Filter(
            should=[
                models.FieldCondition(
                    key="metadata.filename",
                    match=models.MatchValue(value="112年國泰醫療財團法人財務報告.pdf")
                ),
                models.FieldCondition(
                    key="metadata.origin.filename",
                    match=models.MatchValue(value="112年國泰醫療財團法人財務報告.pdf")
                )
            ]
        )
    ),
)
delete_result

In [20]:
data = DataObject(node_text, node_metadatas)
data.text[0]

'國泰金融控股股份有限公司員工國內出差要點\n940405 訂5 訂定\n951031 修訂\n960509 修9 修訂\n980728 修8 修訂\n1020101 修訂\n1030206 修6 修訂\n1030523 修3 修訂\n1040904 修4 修訂\n1070326 修6 修訂\n1090101 修訂\n1120801修1修訂\n1130701修1修訂\n權責單位：人力資源部'

In [4]:
# create a qdrant vectorDB object
vector_db = qdrant_DBConnector("qdrant_withID", recreate=False)

In [22]:
# add data to db
vector_db.upsert_vector(embedded_text, data)

upsert finish


# Test Retrieval with KB_NAME

In [5]:
#query = "去United Arab Emirates出差可以申請多少費用？"
query = "科主管國內出差可以申請多少餐雜宿費？"
embedded_query = get_embeddings(query)
retrieved_kb_name = "test_kb_8d4e59e6"

In [6]:
vector_result_json_with_kbname = vector_db.vector_search_json_with_kb_name(kb_name=retrieved_kb_name, vector=embedded_query, top_k=3)
show_json(vector_result_json_with_kbname)

{
    "chunk_5f6c5e73-9972-4888-8cf7-2fde53576a22": {
        "text": "註：\n1. 本要點所稱「高階主管」係指部室主管(不含)以上，總經理(不含)以下之主管。\n2. 長期出差(出差期間超過 1 個月)，自第 3 個月起，餐雜費依上述標準 4 折計算。\n3. 餐雜費結構比例為餐費、雜費各 50%；出差期間如供餐，依下列比例扣除：早、中、晚餐各 1/3。\n4. 參加國際會議時，如主辦單位指定住宿飯店超出本辦法標準，其宿費依該指定飯店之最之最低最低標準標準 核給。\n5. 部室主管(含(含)以下人員與高階主管(含(含)以上人員同行出國，宿費以投宿同-飯店較低標準之 房型為限，且費用不得超出同行主管宿費標準上限或實際申報額度孰低；餐雜費仍依原職級標級標 準核給。\n6. 出差地若有二處(含(含)以上並分屬不同等級城市或地區，依不同地點之停留期間分開計算。\n7. 匯率換算標準：依出國前-星期內賣匯水單匯率或出國前-營業日國泰世華銀行換匯幣別 (現(現鈔)收盤賣出匯率計算。\n附表二：飛機艙別及其他交通工具搭乘標準\n\n總經理(含)以上, 搭乘艙等 = 頭等或相當之座(艙)位. 高階主管, 搭乘艙等 = 商務或相當之座(艙)位. 其他人員(註), 搭乘艙等 = 經濟(標準)或相當之座(艙)位\n註：部室級主管搭乘歐美航線得選擇商務或相當之座(艙(艙)位)位；部室主管(不含)以下人員搭員搭乘歐美航線得選 擇不限價位之經濟艙",
        "metadata": {
            "filename": "國泰金控員工國外出差要點_c0a8eb7a.pdf",
            "table_ref": [
                "國泰金控員工國外出差要點_c0a8eb7a-table-3.png"
            ],
            "image_ref": [],
            "page_ref": [
                4,
                5
            ],
            "file_id": "c0a8eb7a",
            "kb_name": "test_kb

In [7]:
bm25_retrieve_result_with_kbname = bm25_retrieval_with_kb_name(vector_db, kb_name=retrieved_kb_name, query=query, top_k=3)
show_json(bm25_retrieve_result_with_kbname)

Building prefix dict from /Users/yoyo/Documents/國泰/BM25/dict.txt.big ...
Loading model from cache /var/folders/v7/_g3b28s51m532s03dhwffy9c0000gn/T/jieba.u938b08ca6a75588702a8549caaec8c7e.cache
Loading model cost 0.763 seconds.
Prefix dict has been built successfully.


{
    "chunk_e213441f-dba3-44cd-9473-a74a294d691e": {
        "text": "附表三：投保標準\n單位：新台幣\n, 每人投保保額 = 意外(傷害)   死殘保額. , 每人投保保額 = 傷害醫療限額. , 每人投保保額 = 海外突發  疾病醫療(含燒燙傷)  保險限額(註). , 每人投保保額 = 海外旅遊  不便保險. 總經理(含)以上, 每人投保保額 = 2,000 萬元. 總經理(含)以上, 每人投保保額 = 意外(傷害)   死殘保額 10%. 總經理(含)以上, 每人投保保額 = 意外(傷害)   死殘保額 10%. 總經理(含)以上, 每人投保保額 = 販售之海外  旅遊不便保. 高階主管, 每人投保保額 = 1,600 萬元. 高階主管, 每人投保保額 = 意外(傷害)   死殘保額 10%. 高階主管, 每人投保保額 = 意外(傷害)   死殘保額 10%. 高階主管, 每人投保保額 = 販售之海外  旅遊不便保. 部室/科主管, 每人投保保額 = 1200 萬元. 部室/科主管, 每人投保保額 = 意外(傷害)   死殘保額 10%. 部室/科主管, 每人投保保額 = 意外(傷害)   死殘保額 10%. 部室/科主管, 每人投保保額 = 販售之海外  旅遊不便保. 及相當職級, 每人投保保額 = 1200 萬元. 及相當職級, 每人投保保額 = 意外(傷害)   死殘保額 10%. 及相當職級, 每人投保保額 = 意外(傷害)   死殘保額 10%. 及相當職級, 每人投保保額 = 販售之海外  旅遊不便保. 其他人員, 每人投保保額 = 1200 萬元. 其他人員, 每人投保保額 = 意外(傷害)   死殘保額 10%. 其他人員, 每人投保保額 = 意外(傷害)   死殘保額 10%. 其他人員, 每人投保保額 = 販售之海外  旅遊不便保\n- 註 1:以保費較低組合為限。\n註 2:依:依外交部領事局或行政院大陸委員會所公告旅遊警示燈號，出差地列為黃色警示區域以上(含(含橙色 及紅色)者)者，得提高投保標準-層級，惟紅色及橙色警示地區建議不宜前往。",
        "metadata": {
            "filename": "國泰金控員工國外出差要點_c

# Retrieved From Whole Collection (OLD FUNCTION)

In [8]:
query = "去United Arab Emirates出差可以申請多少費用？"

print("search for:", query)
bm25_retrieve_result = bm25_retrieval(vector_db, query, top_k=3)
show_json(bm25_retrieve_result)

search for: 去United Arab Emirates出差可以申請多少費用？
{
    "chunk_aac8ca31-166a-403e-809f-c5bc849fdcde": {
        "text": "柬埔寨(Cambodia), 協理級(含) 以下人員 宿費 餐雜費.餐雜費 = 85. 寮國(Laos), 總經理級以上主管 宿費 餐雜費.宿費 = 檢據實支. 寮國(Laos), 總經理級以上主管 宿費 餐雜費.餐雜費 = 90或檢據實支. 寮國(Laos), 高階主管.宿費 = 240. 寮國(Laos), 高階主管.餐雜費 = 90. 寮國(Laos), 協理級(含) 以下人員 宿費 餐雜費.宿費 = 135. 寮國(Laos), 協理級(含) 以下人員 宿費 餐雜費.餐雜費 = 85. 越南(Vietnam), 總經理級以上主管 宿費 餐雜費.宿費 = 檢據實支. 越南(Vietnam), 總經理級以上主管 宿費 餐雜費.餐雜費 = 90或檢據實支. 越南(Vietnam), 高階主管.宿費 = 240. 越南(Vietnam), 高階主管.餐雜費 = 90. 越南(Vietnam), 協理級(含) 以下人員 宿費 餐雜費.宿費 = 175. 越南(Vietnam), 協理級(含) 以下人員 宿費 餐雜費.餐雜費 = 85. 阿拉伯聯合大公國 (United Arab Emirates), 總經理級以上主管 宿費 餐雜費.宿費 = 檢據實支. 阿拉伯聯合大公國 (United Arab Emirates), 總經理級以上主管 宿費 餐雜費.餐雜費 = 160或檢據實支. 阿拉伯聯合大公國 (United Arab Emirates), 高階主管.宿費 = 395. 阿拉伯聯合大公國 (United Arab Emirates), 高階主管.餐雜費 = 160. 阿拉伯聯合大公國 (United Arab Emirates), 協理級(含) 以下人員 宿費 餐雜費.宿費 = 315. 阿拉伯聯合大公國 (United Arab Emirates), 協理級(含) 以下人員 宿費 餐雜費.餐雜費 = 120. 以色列(Israel), 總經理級以上主管 宿費 餐雜費.宿費 = 檢據實支. 以色列(Israel), 總經理級以上主管 宿

In [9]:
embedded_query = get_embeddings(query)
vector_result_json = vector_db.vector_search_json(embedded_query, 3)
print("search for:", query)

show_json(vector_result_json)

search for: 去United Arab Emirates出差可以申請多少費用？
{
    "chunk_0d835f67-fa96-4f14-8b73-0632a3919900": {
        "text": "第六條(其他費用)\n辦理出國必備之包之包括護照費、簽證費、機場服務費等，得檢據請領；若於出差國家轉搭國內線 航班，其票價未含行李費者，該行李費得列入手續費，惟以1件1件或20公斤為限。其他未列於本 辦法中費用，如因業務需要，以簽准內容為限，實報實銷。奉派出國人員交際應酬費用除奉准 由公司開支之部份外，-概自理。",
        "metadata": {
            "filename": "國泰金控員工國外出差要點_c0a8eb7a.pdf",
            "table_ref": [],
            "image_ref": [],
            "page_ref": [
                1
            ],
            "file_id": "c0a8eb7a",
            "kb_name": "test_kb_8d4e59e6",
            "kb_id": "8d4e59e6"
        },
        "rank": 0,
        "score": 0.59707326
    },
    "chunk_2a42e834-1954-4f88-ae81-49a4cbc0d916": {
        "text": "第三條(餐(餐雜宿費標準)\n員工跨境出差之宿費及餐雜費悉費悉依附表-標準支給之。\n第四條(飛(飛機艙別及其他交通工具搭乘標準及交通費)\n奉派出國人員往返機票費及其他交通工具費用准予按實報支，其標準如附表二。\n在國外必須之交通，應依經奉准旅程表之路程為之，其交通費須檢附收據按實報支，自辦公 (住(住宿)處)處往返國際機場且來回車資，副總經理(含(含)以上實報實銷，協理級(含(含)以下同仁當仁當次出 差往返國內外機場之費用總計上限新限新台幣六千六千元得檢據核銷。",
        "metadata": {
            "filename": "國泰金

In [10]:
#old
#hybrid_result = rrf([vector_result_json, bm25_retrieve_result])

hybrid_result = rrf([vector_result_json_with_kbname, bm25_retrieve_result_with_kbname])

print(json.dumps(hybrid_result, indent=4, ensure_ascii=False))

{
    "chunk_5f6c5e73-9972-4888-8cf7-2fde53576a22": {
        "score": 1.0,
        "text": "註：\n1. 本要點所稱「高階主管」係指部室主管(不含)以上，總經理(不含)以下之主管。\n2. 長期出差(出差期間超過 1 個月)，自第 3 個月起，餐雜費依上述標準 4 折計算。\n3. 餐雜費結構比例為餐費、雜費各 50%；出差期間如供餐，依下列比例扣除：早、中、晚餐各 1/3。\n4. 參加國際會議時，如主辦單位指定住宿飯店超出本辦法標準，其宿費依該指定飯店之最之最低最低標準標準 核給。\n5. 部室主管(含(含)以下人員與高階主管(含(含)以上人員同行出國，宿費以投宿同-飯店較低標準之 房型為限，且費用不得超出同行主管宿費標準上限或實際申報額度孰低；餐雜費仍依原職級標級標 準核給。\n6. 出差地若有二處(含(含)以上並分屬不同等級城市或地區，依不同地點之停留期間分開計算。\n7. 匯率換算標準：依出國前-星期內賣匯水單匯率或出國前-營業日國泰世華銀行換匯幣別 (現(現鈔)收盤賣出匯率計算。\n附表二：飛機艙別及其他交通工具搭乘標準\n\n總經理(含)以上, 搭乘艙等 = 頭等或相當之座(艙)位. 高階主管, 搭乘艙等 = 商務或相當之座(艙)位. 其他人員(註), 搭乘艙等 = 經濟(標準)或相當之座(艙)位\n註：部室級主管搭乘歐美航線得選擇商務或相當之座(艙(艙)位)位；部室主管(不含)以下人員搭員搭乘歐美航線得選 擇不限價位之經濟艙",
        "metadata": {
            "filename": "國泰金控員工國外出差要點_c0a8eb7a.pdf",
            "table_ref": [
                "國泰金控員工國外出差要點_c0a8eb7a-table-3.png"
            ],
            "image_ref": [],
            "page_ref": [
                4,
                5
            ],
            "file_id": "c0a8eb7a",
         

In [11]:
query = "去United Arab Emirates出差可以申請多少費用？"
#query = "重要會計項目中，現金及約當現金的銀行存款是多少？"
retrieved_kb_name = "test_kb_8d4e59e6"

#print(format_rag_output(reranker(query, hybrid_retriever(vector_db, query, top_k=35), threshold=0.45)))

reranked_list = reranker(query, hybrid_retriever_with_kbname(vector_db, retrieved_kb_name, query, 20), threshold=0.45)
rag_docs = format_rag_output(reranked_list)
print(rag_docs)

print()
#for chunk in reranker(query, hybrid_retriever(query, 20)):
for chunk in reranker(query, hybrid_retriever_with_kbname(vector_db, retrieved_kb_name, query, top_k=35)):
    print(chunk)
    print()

Document 1，國泰金控員工國外出差要點_c0a8eb7a:
費.費 = 1, 出差地.費用類別 = 新加坡 (Singapore), 高階主管及  總經理級以上主管.宿費 = 400, 高階主管及  總經理級以上主管.雜費 = 檢據實支, 協理級(含)以下人員  宿費 雜費.宿費 = 320, 協理級(含)以下人員  宿費 雜費.雜費 = 20
費.費 = 2, 出差地.費用類別 = 香港(Hong Kong), 高階主管及  總經理級以上主管.宿費 = 350, 高階主管及  總經理級以上主管.雜費 = 檢據實支, 協理級(含)以下人員  宿費 雜費.宿費 = 280, 協理級(含)以下人員  宿費 雜費.雜費 = 18
費.費 = 3, 出差地.費用類別 = 菲律賓 (Philippines), 高階主管及  總經理級以上主管.宿費 = 260, 高階主管及  總經理級以上主管.雜費 = 檢據實支, 協理級(含)以下人員  宿費 雜費.宿費 = 210, 協理級(含)以下人員  宿費 雜費.雜費 = 13
費.費 = 4, 出差地.費用類別 = 泰國 (Thailand), 高階主管及  總經理級以上主管.宿費 = 245, 高階主管及  總經理級以上主管.雜費 = 檢據實支, 協理級(含)以下人員  宿費 雜費.宿費 = 200, 協理級(含)以下人員  宿費 雜費.雜費 = 13
費.費 = 5, 出差地.費用類別 = 馬來西亞 (Malaysia), 高階主管及  總經理級以上主管.宿費 = 260, 高階主管及  總經理級以上主管.雜費 = 檢據實支, 協理級(含)以下人員  宿費 雜費.宿費 = 190, 協理級(含)以下人員  宿費 雜費.雜費 = 12
費.費 = 6, 出差地.費用類別 = 尼 (Indonesia), 高階主管及  總經理級以上主管.宿費 = 260, 高階主管及  總經理級以上主管.雜費 = 檢據實支, 協理級(含)以下人員  宿費 雜費.宿費 = 190, 協理級(含)以下人員  宿費 雜費.雜費 = 12
費.費 = 7, 出差地.費用類別 = 緬甸 (Burma, 高階主管及  總經理級以上主管.宿費 = 260, 高階主管及  總經理級以上主管.雜費 = 檢據實支, 協理級(含)以下人員  宿費 雜費.宿費 =

In [12]:
# Job instruction
instruction = """
你是台灣國泰集團的聊天機器人秘書，專門為用戶提供公司內外文件內容的解析和答疑，
你的任務是根據你獲得的「參考文件」，對「用戶問題」段落的問題進行回答。

請務必根據「參考文件」中的具體資訊作答，並注意以下要求：
1. 若某些文件內容對回答無幫助，可以忽略，不採用。
2. 若文件內容對回答有幫助，不要忽略任何一絲細節。
3. 回答應簡潔、明確，避免冗長，僅提取關鍵資訊。
4. 若參考文件無法提供答案，請直接回答「我無法根據現有資料回答這個問題」，並不要自行補充。

嚴格使用繁體中文，避免英文或簡體中文。
"""

# User input
#input_text = "去中國大陸出差可以申請多少費用？"
input_text = "去義大利出差可以申請多少費用？"
#input_text = "去智利出差可以申請多少費用？"
#input_text = "科主管國內出差可以申請多少餐雜宿費？"
#input_text = "商品貨幣是什麼？"
#input_text = "系統開發之安全管理，應包含哪些項目？"
#input_text = "國泰優惠APP 後台維護系統，有什麼功能？"
#input_text = "國泰優惠APP 後台維護系統，功能權限申請流程為何？" 
#input_text = "國泰優惠 APP後台維護系統使用者權限表，有什麼欄位？"
#input_text = "金融科技創新業務之立案及概念測試流程為何？"  
#input_text = "設計阿發時有什麼背景色彩限制？" # Fail? on picture detail explaination, but did catch the words on pics
#input_text = "根據111年度4-6月份捐款及捐贈物資徵信名冊，誰捐了鵝肉湯？" # doc14 fail on parsing compact table index, but seems like it does not interfere with llm answer
#input_text = "根據111年度4-6月份捐款及捐贈物資徵信名冊，梅力化學工業有限公司做了什麼？" # same as above
#input_text = "重要會計項目中，現金及約當現金的銀行存款，111年與112年分別是多少？" # loose structured table, success
#input_text = "幫我統整一下不動產、廠房及設備中，成本，折舊以及淨帳面金額數字" # loose structured table, poor recognize result, but success seemingly
#input_text = "請給我應收帳款(淨額)之帳齡分析的內容"
#input_text = "請給我112年，藥品進貨交易對象的名稱，金額及比率"
#input_text = "113年工作計畫有什麼社區服務相關的內容嗎？"
#input_text = "幫我總結一下113年度經費預算與上年度的比較差異"

# KB name
retrieved_kb_name = "test_kb_8d4e59e6"

# RAG retrieved documents
#reranked_list = reranker(input_text, hybrid_retriever(vector_db, input_text, 20), threshold=0.45)
#rag_docs = format_rag_output(reranked_list)

reranked_list = reranker(input_text, hybrid_retriever_with_kbname(vector_db, retrieved_kb_name, input_text, 20), threshold=0.45)
rag_docs = format_rag_output(reranked_list)

# Prompt template
prompt = f"""
# 任務
{instruction}

# 參考文件
{rag_docs}

# 用戶問題
{input_text}
"""

print("==== Prompt ====")
print(prompt)
print("================")

# llm calling
if len(rag_docs) != 0:
    response = get_completion(prompt, llm)
else:
    response = "YAh, you retrieved NOTHING!"
print(response)

==== Prompt ====

# 任務

你是台灣國泰集團的聊天機器人秘書，專門為用戶提供公司內外文件內容的解析和答疑，
你的任務是根據你獲得的「參考文件」，對「用戶問題」段落的問題進行回答。

請務必根據「參考文件」中的具體資訊作答，並注意以下要求：
1. 若某些文件內容對回答無幫助，可以忽略，不採用。
2. 若文件內容對回答有幫助，不要忽略任何一絲細節。
3. 回答應簡潔、明確，避免冗長，僅提取關鍵資訊。
4. 若參考文件無法提供答案，請直接回答「我無法根據現有資料回答這個問題」，並不要自行補充。

嚴格使用繁體中文，避免英文或簡體中文。


# 參考文件
Document 1，國泰金控員工國外出差要點_c0a8eb7a.pdf:
. 盧森堡(Luxembourg), 協理級(含) 以下人員 宿費 餐雜費.宿費 = 380. 盧森堡(Luxembourg), 協理級(含) 以下人員 宿費 餐雜費.餐雜費 = 165. 瑞士(Switzerland), 總經理級以上主管 宿費 餐雜費.宿費 = 檢據實支. 瑞士(Switzerland), 總經理級以上主管 宿費 餐雜費.餐雜費 = 165或檢據實支. 瑞士(Switzerland), 高階主管.宿費 = 405. 瑞士(Switzerland), 高階主管.餐雜費 = 165. 瑞士(Switzerland), 協理級(含) 以下人員 宿費 餐雜費.宿費 = 325. 瑞士(Switzerland), 協理級(含) 以下人員 宿費 餐雜費.餐雜費 = 125. 丹麥(Denmark), 總經理級以上主管 宿費 餐雜費.宿費 = 檢據實支. 丹麥(Denmark), 總經理級以上主管 宿費 餐雜費.餐雜費 = 140或檢據實支. 丹麥(Denmark), 高階主管.宿費 = 345. 丹麥(Denmark), 高階主管.餐雜費 = 140. 丹麥(Denmark), 協理級(含) 以下人員 宿費 餐雜費.宿費 = 275. 丹麥(Denmark), 協理級(含) 以下人員 宿費 餐雜費.餐雜費 = 105 瑞典(Sweden), 1 = 檢據實支. 瑞典(Sweden), 2 = 160或檢據實支. 瑞典(Sweden), 3 = 390. 瑞典(Sweden), 4 = 160. 瑞典(Sweden), 5

In [ ]:
text = """1. 摘要圖片的主要內容：
這是一個關於國泰金融控股公司後台維護系統的管理作業流程圖，主要描述了申請、審核、承辦作業及驗收各階段的處理步驟。

2. 擷取重要概念、重點事項或關鍵字：
- 申請人
- 所屬單位
- 生活金融科
- 會員個人資料
- 資料管理單位
- 功能權限
- 帳號對應權限代碼
- 驗收

3. 結構化資訊描述：
- **申請階段**
  1. 申請人提出申請需求
  2. 申請人所屬單位主管審核

- **審核階段**
  3. 生活金融科會辦審核
     - 是否涉及會員個人資料
       - 是：會簽資料所屬單位/資料管理單位/其他相關單位
       - 否：金控資訊部分派申辦單

- **承辦作業階段**
  6. 生活金融科帳號權限設定
     - 功能權限申請是否一致
       - 是：設定帳號對應權限代碼
       - 否：新增代碼對應功能權限

- **驗收階段**
  9. 驗收結案
"""
len(tokenizer.tokenize(text))